In [ ]:
#default_exp rural_only

In [1]:
import pandas as pd

In [2]:
%%time
infile = 'data/df_2017_OMB_Census_HRSA_FAR.csv'
df = pd.read_csv(infile,dtype=object)

CPU times: user 2min 24s, sys: 15.2 s, total: 2min 39s
Wall time: 2min 39s


In [3]:
# Create a dataframe with only the records that were flagged as rural by any of the 5 criteria.

In [4]:
df['FAR Level'].fillna('-1',inplace=True)
df['FAR Level'] = df['FAR Level'].astype(float)
df[['rural_OMB','rural_Census_general','rural_Census','rural_HRSA']] = \
    df[['rural_OMB','rural_Census_general','rural_Census','rural_HRSA']].astype(int) 

In [5]:
df['rural_OMB'].value_counts()

0    13849980
1      883457
Name: rural_OMB, dtype: int64

In [6]:
df['rural_Census_general'].value_counts()

0    13042415
1     1691022
Name: rural_Census_general, dtype: int64

In [7]:
df['rural_Census'].value_counts()

0    12705736
1     2027701
Name: rural_Census, dtype: int64

In [8]:
df['rural_HRSA'].value_counts()

0    13740765
1      992672
Name: rural_HRSA, dtype: int64

In [9]:
df['FAR Level'] = df['FAR Level'].astype(int)
df['FAR Level'].value_counts()

 0    12625580
-1     1467714
 1      229933
 2      168683
 4      126095
 3      115432
Name: FAR Level, dtype: int64

In [10]:
df['rural_FAR'] = df['FAR Level'].apply(lambda x: 1 if x >= 1 else 0)

In [11]:
df['rural_FAR'].value_counts()

0    14093294
1      640143
Name: rural_FAR, dtype: int64

In [32]:
%%time
df_rural_any_with_FAR = df[(df['rural_OMB']  == 1) | (df['rural_Census_general'] == 1) | (df['rural_Census'] == 1) |
             (df['rural_HRSA'] == 1) | (df['rural_FAR'] == 1)].copy()
len(df_rural_any_with_FAR)

CPU times: user 14.8 s, sys: 2 s, total: 16.8 s
Wall time: 6.78 s


3417035

In [33]:
%%time
df_rural_any = df[(df['rural_OMB']  == 1) | (df['rural_Census_general'] == 1) | (df['rural_Census'] == 1) |
             (df['rural_HRSA'] == 1)].copy()
len(df_rural_any)

CPU times: user 13.9 s, sys: 1.38 s, total: 15.2 s
Wall time: 5.94 s


3187230

In [34]:
df_rural_all_with_FAR = df[(df['rural_OMB']  == 1) & (df['rural_Census_general'] == 1) & (df['rural_Census'] == 1) &
             (df['rural_HRSA'] == 1) & (df['rural_FAR'] == 1)].copy()
len(df_rural_all_with_FAR)

196439

In [35]:
df_rural_all = df[(df['rural_OMB']  == 1) & (df['rural_Census_general'] == 1) & (df['rural_Census'] == 1) &
             (df['rural_HRSA'] == 1)].copy()
len(df_rural_all)

551006

In [36]:
round(len(df_rural_any_with_FAR)/len(df) * 100,2)

23.19

In [37]:
round(len(df_rural_any)/len(df) * 100,2)

21.63

In [38]:
round(len(df_rural_all_with_FAR)/len(df) * 100,2)

1.33

In [39]:
round(len(df_rural_all)/len(df) * 100,2)

3.74

In [20]:
# So the dataframe above called df_and_rural contains all records that qualify as rural on ALL the criteria
# except FAR, which is too restrictive. They make up 3.74% of the InfoGroup records (n == 551,006 in all 50 states
# and DC).

# Over 23% of all records (dataframe df_or_rural) are flagged as rural by ANY of the criteria (including FAR, 
# which, unexpectedly, adds about 230,000 records to the total of 3,187,230 that are not flagged rural by any 
# other criterion. Why? Something about zip codes...).

In [21]:
# What is special about those ~230,000 enterprises?
df_diff = df_or_rural.merge(df_or_rural_2,on='ABI',how='outer',indicator=True)

In [22]:
df_diff['_merge'].value_counts()

both          3187230
left_only      229805
right_only          0
Name: _merge, dtype: int64

In [23]:
df_dff = df_diff[df_diff['_merge'] == 'left_only']

In [24]:
df_dff['FAR Level_x'].value_counts() 
# total below 229,805, which is 36% of the total of 640,143 FAR-rural enterprises.
# Why would so many FAR-rural enterprises not be considered rural by any other criterion?

1    136008
2     88002
3      5730
4        65
Name: FAR Level_x, dtype: int64

In [31]:
# It is best to do away with the FAR criterion completely because it seems not to correlate well with units
# that are the result of defining urban/rural explicitly. FAR is a measure of remoteness and it implies rurality 
# but it is not a comprehensive standard. It is evident that some remote locations (by the FAR definition) are 
# in urban areas.

# Eliminating the FAR codes, we have 3,187,230 (21.63%) of the InfoGroup records in at least one of the other
# rurality criteria. We have 551,006 (3.74%) of the InfoGroup records in all of the other rurality criteria.
# Dataframes:
# df_rural_any: 3,187,230 (21.63%)
# df_rural_all: 551,006 (3.74%)

In [52]:
pd.options.display.max_rows = 4000
df_rural_all['NAICS8 Descriptions'].value_counts(normalize=True) * 100

LEGISLATIVE BODIES                                    6.222093
RELIGIOUS ORGANIZATIONS                               5.473796
COMMERCIAL BANKING                                    3.922933
FULL-SERVICE RESTAURANTS                              2.984515
ELEMENTARY & SECONDARY SCHOOLS                        2.433536
OFFICES OF PHYSICIANS (EXC MENTAL HEALTH SPECS)       2.405664
ALL OTHER MISCELLANEOUS CROP FARMING                  2.208530
UNCLASSIFIED ESTABLISHMENTS                           2.181766
INSURANCE AGENCIES & BROKERAGES                       1.859485
OFFICES OF ALL OTHER MISC HEALTH PRACTITIONERS        1.631710
NEW SINGLE-FAMILY HSNG CONSTR (EXC FOR-SALE BLDRS)    1.583718
BEAUTY SALONS                                         1.424424
GENERAL AUTOMOTIVE REPAIR                             1.347822
OTHER GASOLINE STATIONS                               1.284510
POSTAL SERVICE                                        1.261806
OFFICES OF REAL ESTATE AGENTS & BROKERS               1

In [54]:
df_rural_all[['Company','Primary NAICS Code']][df_rural_all['NAICS8 Descriptions']=='LEGISLATIVE BODIES']

,Company,Primary NAICS Code
12532889,NANTUCKET TEEN CTR,92112006
12537479,SANDWICH TOWN CLERK,92112006
12537482,SELECTMEN'S OFFICE,92112006
12537511,SANDWICH TRANSFER STATION,92112006
12541543,MOULTONBOROUGH ROAD AGENT,92112006
...,...,...
14733386,WRANGELL CITY LIGHT PLANT,92112006
14733392,WRANGELL SEARCH & RESCUE,92112006
14733393,FEDERAL AVIATION ADMIN,92112009
14733420,JUDGE'S CHAMBERS,92112008


In [55]:
pd.options.display.max_rows = 4000
df_rural_any['NAICS8 Descriptions'].value_counts(normalize=True) * 100

OFFICES OF PHYSICIANS (EXC MENTAL HEALTH SPECS)       4.080883
RELIGIOUS ORGANIZATIONS                               3.875972
LEGISLATIVE BODIES                                    3.714686
FULL-SERVICE RESTAURANTS                              3.320119
COMMERCIAL BANKING                                    3.297366
UNCLASSIFIED ESTABLISHMENTS                           3.214106
INSURANCE AGENCIES & BROKERAGES                       1.896213
OFFICES OF ALL OTHER MISC HEALTH PRACTITIONERS        1.891298
ELEMENTARY & SECONDARY SCHOOLS                        1.793805
OFFICES OF LAWYERS                                    1.674516
NEW SINGLE-FAMILY HSNG CONSTR (EXC FOR-SALE BLDRS)    1.651411
OFFICES OF REAL ESTATE AGENTS & BROKERS               1.623105
BEAUTY SALONS                                         1.374889
OFFICES OF DENTISTS                                   1.308479
PLUMBING HTG & AIR-CONDITIONING CONTRACTORS           1.269290
GENERAL AUTOMOTIVE REPAIR                             1

In [56]:
df['NAICS8 Descriptions'].value_counts(normalize=True) * 100

OFFICES OF PHYSICIANS (EXC MENTAL HEALTH SPECS)       7.560790
FULL-SERVICE RESTAURANTS                              3.899111
UNCLASSIFIED ESTABLISHMENTS                           3.699691
OFFICES OF LAWYERS                                    3.166161
COMMERCIAL BANKING                                    3.071177
OFFICES OF ALL OTHER MISC HEALTH PRACTITIONERS        2.865309
INSURANCE AGENCIES & BROKERAGES                       2.455930
RELIGIOUS ORGANIZATIONS                               2.451404
LEGISLATIVE BODIES                                    2.242302
OFFICES OF REAL ESTATE AGENTS & BROKERS               2.079718
OFFICES OF DENTISTS                                   2.039399
BEAUTY SALONS                                         1.489430
OTHER INDIVIDUAL & FAMILY SERVICES                    1.420165
NEW SINGLE-FAMILY HSNG CONSTR (EXC FOR-SALE BLDRS)    1.219894
ELEMENTARY & SECONDARY SCHOOLS                        1.207000
OFFICES-PHYSICAL, OCCPTNL/SPEECH THRPSTS/AUDLGSTS     1

In [58]:
# Look at the same distribution in a random sample of InfoGroup data exactly the same size as df_rural_all.
df_small_sample = df.sample(n=551006, random_state=1)
df_small_sample['NAICS8 Descriptions'].value_counts(normalize=True) * 100

OFFICES OF PHYSICIANS (EXC MENTAL HEALTH SPECS)       7.532101
FULL-SERVICE RESTAURANTS                              3.889116
UNCLASSIFIED ESTABLISHMENTS                           3.649638
OFFICES OF LAWYERS                                    3.200062
COMMERCIAL BANKING                                    3.044106
OFFICES OF ALL OTHER MISC HEALTH PRACTITIONERS        2.872997
RELIGIOUS ORGANIZATIONS                               2.455758
INSURANCE AGENCIES & BROKERAGES                       2.454650
LEGISLATIVE BODIES                                    2.241226
OFFICES OF REAL ESTATE AGENTS & BROKERS               2.082313
OFFICES OF DENTISTS                                   2.057552
BEAUTY SALONS                                         1.496738
OTHER INDIVIDUAL & FAMILY SERVICES                    1.416542
NEW SINGLE-FAMILY HSNG CONSTR (EXC FOR-SALE BLDRS)    1.233977
ELEMENTARY & SECONDARY SCHOOLS                        1.223075
OFFICES-PHYSICAL, OCCPTNL/SPEECH THRPSTS/AUDLGSTS     1

In [59]:
# So the random sample of 3.74% has pretty much exactly the same distribution of NAICS values as the full file,
# as expected. But it is quite different from the df_rural_all group of records of the same size. 

In [65]:
fulldf = pd.DataFrame(df['NAICS8 Descriptions'].value_counts(normalize=True) * 100)
fulldf.to_csv('fulldf.csv')

In [66]:
alldf = pd.DataFrame(df_rural_all['NAICS8 Descriptions'].value_counts(normalize=True) * 100)
alldf.to_csv('alldf.csv')

In [67]:
anydf = pd.DataFrame(df_rural_any['NAICS8 Descriptions'].value_counts(normalize=True) * 100)
anydf.to_csv('anydf.csv')